In [5]:
mappings = {"mappings": {
        "legaldocuments": {
            "_source": {
                "enabled": True
            },
            "_all": {
                "enabled": False
            },
            "properties": {
                "title": {
                    "type": "text"
                },
                "content": {
                    "type": "text"
                },
            }
        }
    }}

In [10]:
import os
from glob import glob
import frontmatter
import pypandoc
from elasticsearch import Elasticsearch
from elasticsearch import helpers


os.chdir("../gesetze")
es_client = Elasticsearch("http://localhost:9200", mappings=mappings)

files = []
start_dir = os.getcwd()
pattern   = "[!README]*.md"

for dir,_,_ in os.walk(start_dir):
    files.extend(glob(os.path.join(dir,pattern)))

data = []


for file in files[:100]:
    with open(file) as fn:
        post = frontmatter.load(fn)
        metadata = post.metadata
        content = pypandoc.convert_text(post.content, 'plain', format='md')
        data.append(
            {
                "title": metadata.get("Title"),
                "content": content.replace('\n', ' ').replace('\r', ''),
                "documenttype": "NORM"
            }
        )        
        fn.close()

def doc_generator(data):
    for entry in data:
        yield {
                "_index": 'legaldocuments',
                "_type": "_doc",
                "_source": entry,
            }

helpers.bulk(es_client, doc_generator(data))

[WARNING] Note with key 'F1_818755_BJNR467400021' defined at line 9 column 1 but not used.



(100, [])

In [3]:
len(files)

6636

In [4]:
es_client.indices.delete(index='legaldocuments', ignore=[400, 404])

{'acknowledged': True}